In [60]:
import pandas as pd

def power_used(df_chiller_boiler_raw, start_date, end_date) -> tuple:
    """
    Return boiled and chiller kWh for a given time period from raw data.
    Dates in 'yyyy-MM-dd' (optional hh:mm:ss)
    """
    # Clean data
    df_chiller_boiler_raw.columns = [
        col.replace(' - Extended Trend Log', '').replace(' Extended Trend Log', '').replace(' Trend Log', '').replace(' - Ext', '').replace('thermal', 'Thermal')
        for col in df_chiller_boiler_raw.columns
    ]
    # Convert all columns to float
    exclude_timestamp_col = [col for col in df_chiller_boiler_raw.columns if col != 'Timestamp']
    df_chiller_boiler_raw = df_chiller_boiler_raw.replace(',','', regex=True)
    df_chiller_boiler_raw[exclude_timestamp_col] = df_chiller_boiler_raw[exclude_timestamp_col].apply(pd.to_numeric)

    
    # Filter for only date range 
    date_mask = (df_chiller_boiler_raw['Timestamp'] >= start_date) & (df_chiller_boiler_raw['Timestamp'] < end_date)
    df_chiller_boiler = df_chiller_boiler_raw.loc[date_mask]
    
    totals = {}  # Leaving this here incase raw values need to be inspected
    boiler = chiller = 0
    for column in df_chiller_boiler:
        if column != 'Timestamp':
            column_sum = sum(df_chiller_boiler[column])
            totals[column] = column_sum
            if column.startswith('LTB CH'):
                chiller += column_sum
            elif  column.startswith('LTB  BLR'):
                boiler += column_sum

    # Limit sig figs
    boiler = round(boiler, 2)
    chiller = round(chiller, 2)
    
    return (chiller, boiler)
    
    
    
    
    
        
        

In [61]:
df_chiller_boiler_raw = pd.read_csv('data/more_Data/chillers boilers thermal Feb 23032020.csv', parse_dates=[0],  dayfirst=True)
start_date = '2020-02-01 15:00:40'
end_date = '2020-02-25'


results = power_used(df_chiller_boiler_raw, start_date, end_date) 
print(results)

(135906.13, 846.77)
